In [1]:
import pandas as pd

In [2]:
# Load data
toys_df = pd.read_csv(r"C:\Users\User\Desktop\المشروع الفصلي\AI\my_version\2\toys.csv")
children_df = pd.read_csv(r"C:\Users\User\Desktop\المشروع الفصلي\AI\my_version\2\childrin.csv")

In [3]:
toys_df.head()

,ID,Name,Toy Category,Age,Skill Development,Play Pattern,Educational Value
0,1,Bizzy Beads,Art & Craft,3,Fine Motor Skills,Creative Play,Yes
1,2,Rocket Racer,Remote Control Toys,6,Hand-Eye Coordination,Outdoor Play,Yes
2,3,Jungle Explorer,Educational Toys,4,Cognitive Development,Pretend Play,Yes
3,4,Color Splasher,Drawing & Coloring,2,Imagination,Artistic Play,Yes
4,5,Science Sleuth,Science Kits,8,Scientific Inquiry,Solo Play,Yes


In [4]:
children_df.head()

,ID,Name,Age,Developmental_Stage,Interests_and_Preferences,Challenges_or_Learning_Needs
0,1,Jak Adam,5,Early Childhood,Art and Crafts,Speech Development
1,2,Ava Smith,7,Mid Childhood,Outdoor Exploration,Social Skills
2,3,Liam Johnson,6,Mid Childhood,Science Experiments,Critical Thinking
3,4,Olivia Davis,8,Late Childhood,Reading and Books,Language Fluency
4,5,Noah Miller,9,Late Childhood,Sports and Games,Physical Coordination


VSM

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
#- Index Model : Vector space model
class indexer:

    def __init__(self, documents_df):
      # Concatenate the features into a single text feature
        documents_df['combined_features'] = documents_df['Toy Category'] + ' ' + documents_df['Age'].astype(str) + ' ' + documents_df['Skill Development'] + ' ' + documents_df['Play Pattern']
        # Create a TF-IDF vectorizer
        self.tfidf_vectorizer = TfidfVectorizer()
        #-then finally, fit the documents and transform
        self._index = self.tfidf_vectorizer.fit_transform(documents_df['combined_features'])

    def getindex(self):
        return self._index

    def vectorize(self, sentence):
        if isinstance(sentence,str):
            qry=pd.DataFrame([{"text":sentence}])
        else:
            qry=sentence
        return self.tfidf_vectorizer.transform(qry['text'])


In [9]:
vsm = indexer(toys_df)

In [10]:
print(vsm.tfidf_vectorizer.vocabulary_)

{'art': 4, 'craft': 18, 'fine': 31, 'motor': 48, 'skills': 70, 'creative': 19, 'play': 55, 'remote': 63, 'control': 16, 'toys': 83, 'hand': 35, 'eye': 30, 'coordination': 17, 'outdoor': 52, 'educational': 26, 'cognitive': 12, 'development': 21, 'pretend': 57, 'drawing': 25, 'coloring': 14, 'imagination': 36, 'artistic': 5, 'science': 67, 'kits': 41, 'scientific': 68, 'inquiry': 37, 'solo': 72, 'robotics': 65, 'problem': 58, 'solving': 73, 'interactive': 39, 'physical': 54, 'fitness': 32, 'active': 1, 'puzzles': 60, 'logical': 45, 'thinking': 81, 'musical': 49, 'instruments': 38, 'sensory': 69, 'building': 9, 'blocks': 7, 'creativity': 20, 'construction': 15, 'auditory': 6, 'storybooks': 76, 'language': 43, 'storytelling': 77, 'coding': 11, 'programming': 59, 'logic': 44, 'sports': 75, 'games': 34, 'teamwork': 79, 'pattern': 53, 'recognition': 62, 'dinosaur': 22, 'knowledge': 42, 'space': 74, 'exploration': 28, 'diy': 24, 'fun': 33, 'activity': 2, 'role': 66, 'playing': 56, 'social': 71

In [11]:
class Retriever:

    def retrieve(self, query_vec, index_model):
        cosine_similarities = cosine_similarity(query_vec, index_model.getindex())
        results = pd.DataFrame(
            [{'ID':i+1, 'score':cosine_similarities[0][i]}
            for i in range(len(cosine_similarities[0]))]
        ).sort_values(by=['score'], ascending=False)
        return results[results["score"]>0]

In [12]:
child_id = 4

In [13]:
# Assuming the DataFrame is named 'children_df'
# Accessing the specified columns for the child with the given ID
child_data = children_df.loc[child_id, ['Age', 'Developmental_Stage', 'Interests_and_Preferences', 'Challenges_or_Learning_Needs']]

# Convert the information to text without column names
child_data = f"{child_data['Age']} {child_data['Developmental_Stage']} {child_data['Interests_and_Preferences']} {child_data['Challenges_or_Learning_Needs']}"

print(child_data)


9 Late Childhood Sports and Games Physical Coordination


In [14]:
chold_vector=vsm.vectorize(child_data)
rt = Retriever()
res = rt.retrieve(chold_vector,vsm)
print(res.head())

    ID     score
13  14  0.428786
31  32  0.373170
18  19  0.261146
6    7  0.261146
1    2  0.237924
